In [7]:
%reload_ext autoreload
%autoreload 2

import sys, os, random, copy, collections, time, re, argparse, commands
sys.path.append('src/')
sys.stdout = sys.stderr
import pyhocon
import numpy as np
from pprint import pprint
from logging import FileHandler
import tensorflow as tf
from src.main import Manager
from utils import common, evaluation, tf_utils
from core import models, datasets
args = common.dotDict({
    'checkpoint_path': 'checkpoints/tmp',
    'config_path': 'configs/config',
    'cleanup': True,
    'log_file': None,
    'interactive': True,
    'batch_size': 1,
})

sess = tf.InteractiveSession(graph=tf.Graph())
#sess.as_default()
manager = Manager(args, sess)
config = manager.config

Reading a config from configs/config ...
Restore the config to checkpoints/tmp/config ...
{u'num_train_data': 10000, u'dropout_rate': 0.2, u'num_layers': 1, u'char_vocab_path': u'dataset/embeddings/char_vocab.english.txt', u'decay_frequency': 100, u'w_vocab_size': 30000, u'word_max_len': 20, u'dataset_info': {u'test': {u'path': u'dataset/ubuntu-dialogue/test.csv', u'max_lines': 0}, u'train': {u'path': u'dataset/ubuntu-dialogue/train.csv', u'max_lines': 100000}, u'valid': {u'path': u'dataset/ubuntu-dialogue/valid.csv', u'max_lines': 0}}, u'teacher_forcing': False, u'cell_type': u'GRUCell', u'c_vocab_size': 100, u'max_epoch': 30, u'wbase': True, u'decay_rate': 0.999, u'rnn_type': u'bidirectional_dynamic_rnn', u'c_embedding_size': 8, u'embeddings': [ConfigTree([(u'path', u'dataset/embeddings/glove.840B.300d.txt.filtered'), (u'size', 300), (u'format', u'txt')])], u'utterance_max_len': 20, u'learning_rate': 0.001, u'rnn_size': 50, u'batch_size': 1, u'dataset_type': u'UbuntuDialogueDataset',

In [8]:
batch = manager.debug()

In [9]:
b = batch.next()
w_vocab = manager.w_vocab
c_vocab = manager.c_vocab
sys.stdout = sys.stderr
#print 'w_contexts:', b.w_contexts[0]
#contexts = [w_vocab.id2sent(x, join=True) for x in b.w_contexts[0]]
speaker_changes = b.speaker_changes[0]
#print 'c_contexts:', b.c_contexts[0]
#print '\n'.join([c_vocab.id2sent(x, join=True) for x in b.c_contexts[0]])
contexts = [c_vocab.id2sent(x, join=True) for x in b.c_contexts[0]]

#print 'response:', b.responses[0] 
responses = w_vocab.id2sent(b.responses[0], join=True)

for sc, context in zip(speaker_changes, contexts):
    print sc, context
print 1, responses
 

Loading dataset from dataset/ubuntu-dialogue/train.csv ...
Preprocessing ...
[INFO] 2018-03-23 12:32:48 - load_data: 20.429748 sec
[INFO] 2018-03-23 12:32:48 - load_data: 20.429748 sec
1 splitting xfonts* out of xfree86* . one upload for the rest of the life and that 's it
1 splitting the source package you mean ?
1 yes . same binary packages .
1 I would prefer to avoid it at this stage . this is something that has gone into XSF svn , I assume ?
1 basically each xfree00 upload will not force users to upgrade 000mb of fonts for nothing


In [ ]:
import pandas as pd
sys.stdout = sys.stderr
nrows = 500
train_data = pd.read_csv('dataset/ubuntu-dialogue/train.csv', nrows=nrows)
valid_data = pd.read_csv('dataset/ubuntu-dialogue/valid.csv', nrows=nrows)
data = train_data
batch_size=2
utterance_max_len = 20
context_max_len = 4
_PAD = 0
_EOU = '__eou__'
_EOT = '__eot__'
_URL = '_URL'

import random
_id=random.randint(0, nrows)
from core.datasets import UbuntuDialogueDataset
#vocab = UbuntuDialogueDataset(config.dataset_info.train, config.vocab_size, config.lowercase)
vocab= UbuntuDialogueDataset.create_vocab_from_data(config.dataset_info.train, config.vocab_size, config.lowercase)
dataset = UbuntuDialogueDataset(config.dataset_info, vocab)

# if 'Utterance' in data:
#     context = [preprocess(x) for x in data['Context']]
#     response = [x for x in data['Utterance']]
#     print context[_id]
#     print response[_id]
# else:
#     context = [preprocess(x) for x in data['Context']]
#     response = [x for x in data['Ground Truth Utterance']]
#     n_distractors = 8
#     distractors = [] 
#     for i in xrange(n_distractors):
#         distractors.append([preprocess_utterance(x) for x in data['Distractor_%d' % i]])
#     distractors = list(zip(*distractors))
#     print 'ID', _id
#     print 'Context:', context[_id]
#     print 'Response:', response[_id]
#     for i, d in enumerate(distractors[_id]):
#         print 'Distractor_%d:' % i, d